In [ ]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:8 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Reading package lists... Done


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Beauty_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Beauty_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+---------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|          review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+---------------------+-----------+
|         US|    1797882|R3I2DHQBR577SS|B001ANOOOE|       2102612|The Naked Bee Vit...|          Beauty|          5|            0|          0|   N|                Y|          Five Stars| Love this, excell...| 2015-08-31|
|         US|   18381298|R1QNE9NQFJC2Y4|B0016J22EQ|     106393691|Alba Botanica Sun...|          Beauty|          5|

In [ ]:
# Create the vine_table. DataFrame
# vine_df = df.select([])
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3I2DHQBR577SS|          5|            0|          0|   N|                Y|
|R1QNE9NQFJC2Y4|          5|            0|          0|   N|                Y|
|R3LIDG2Q4LJBAO|          5|            0|          0|   N|                Y|
|R3KSZHPAEVPEAL|          5|            0|          0|   N|                Y|
| RAI2OIG50KZ43|          5|            0|          0|   N|                Y|
|R1R30FA4RB5P54|          4|            0|          0|   N|                Y|
|R30IJKCGJBGPJH|          5|            0|          0|   N|                Y|
|R18GLJJPVQ1OVH|          5|            0|          0|   N|                Y|
| R8TVYIJXLYJT0|          5|            0|          0|   N|                Y|
|R1CJGF6M3PVHEZ|          1|            0|          2|   N|     

In [ ]:
vine_df.dtypes

[('review_id', 'string'),
 ('star_rating', 'string'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string')]

In [ ]:
df1 = vine_df.filter(vine_df.total_votes >=20)
df1.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2QRUE9REK8OUC|          5|           18|         23|   N|                Y|
|R2POXRW6PXHXZG|          5|           25|         30|   N|                Y|
| RZN43XRZ89IIJ|          3|           10|         27|   N|                N|
|R29Q748WSHZ3SN|          5|           23|         24|   N|                Y|
| R46UL5G5HEPRZ|          5|           18|         20|   N|                N|
|R2UQHSNWU6WTZX|          5|           74|         76|   N|                N|
|R2HSRI3D6E2M9Y|          5|           69|         71|   N|                Y|
|R17ARFSDV555EW|          1|           29|         29|   N|                Y|
|R3GL5156FFEDQA|          5|           20|         21|   N|                Y|
|R29KQJC9PIYWQS|          5|           25|         26|   N|     

In [ ]:
df1.dtypes

[('review_id', 'string'),
 ('star_rating', 'string'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string')]

In [ ]:
df2 = df1.filter((df1.helpful_votes / df1.total_votes) >=0.5)
df2.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2QRUE9REK8OUC|          5|           18|         23|   N|                Y|
|R2POXRW6PXHXZG|          5|           25|         30|   N|                Y|
|R29Q748WSHZ3SN|          5|           23|         24|   N|                Y|
| R46UL5G5HEPRZ|          5|           18|         20|   N|                N|
|R2UQHSNWU6WTZX|          5|           74|         76|   N|                N|
|R2HSRI3D6E2M9Y|          5|           69|         71|   N|                Y|
|R17ARFSDV555EW|          1|           29|         29|   N|                Y|
|R3GL5156FFEDQA|          5|           20|         21|   N|                Y|
|R29KQJC9PIYWQS|          5|           25|         26|   N|                Y|
| RYWUJT2A33VJH|          5|           20|         22|   N|     

In [ ]:
df2.dtypes

[('review_id', 'string'),
 ('star_rating', 'string'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string')]

In [ ]:
paid_df = df2.filter(df2.vine == "Y")
paid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R106V6GUNWRSSS|          5|          857|        889|   Y|                N|
|R37N8KZS48W36A|          5|           36|         37|   Y|                N|
|R1LNEQ2MQE03PD|          5|           49|         66|   Y|                N|
|R1ECPWPUNK36ES|          5|           29|         31|   Y|                N|
|R16BUM7UQZLOM6|          3|          107|        112|   Y|                N|
| R7NWIOCA5RVPR|          5|           19|         26|   Y|                N|
|R3NTGOCUZB33JK|          5|           40|         47|   Y|                Y|
| RQX3RHGMG7UBN|          5|          186|        200|   Y|                N|
|R3E6QYKZDST042|          4|           33|         39|   Y|                N|
|R2GR106PWUN85O|          4|           36|         39|   Y|     

In [ ]:
unpaid_df = df2.filter(df2.vine == "N")
unpaid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2QRUE9REK8OUC|          5|           18|         23|   N|                Y|
|R2POXRW6PXHXZG|          5|           25|         30|   N|                Y|
|R29Q748WSHZ3SN|          5|           23|         24|   N|                Y|
| R46UL5G5HEPRZ|          5|           18|         20|   N|                N|
|R2UQHSNWU6WTZX|          5|           74|         76|   N|                N|
|R2HSRI3D6E2M9Y|          5|           69|         71|   N|                Y|
|R17ARFSDV555EW|          1|           29|         29|   N|                Y|
|R3GL5156FFEDQA|          5|           20|         21|   N|                Y|
|R29KQJC9PIYWQS|          5|           25|         26|   N|                Y|
| RYWUJT2A33VJH|          5|           20|         22|   N|     

In [ ]:
# paid total number of reviews
total_paid_reviews = paid_df.count()
total_paid_reviews

647

In [ ]:
# paid 5-star reviews
paid_five_star_reviews = paid_df.filter(paid_df.star_rating == 5).count()
paid_five_star_reviews

229

In [ ]:
# paid 5-star reviews percentage
paid_five_star_percent = (paid_five_star_reviews / total_paid_reviews) * 100
paid_five_star_percent

35.394126738794434

In [ ]:
# unpaid total number of reviews
total_unpaid_reviews = unpaid_df.count()
total_unpaid_reviews

74113

In [ ]:
# unpaid 5-star reviews
unpaid_five_star_reviews = unpaid_df.filter(unpaid_df.star_rating == 5).count()
unpaid_five_star_reviews

43217

In [ ]:
# unpaid 5-star reviews percentage
unpaid_five_star_percent = (unpaid_five_star_reviews / total_unpaid_reviews) * 100
unpaid_five_star_percent

58.312306882733125

In [53]:
# paid 1-star reviews
paid_one_star_reviews = paid_df.filter(paid_df.star_rating == 1).count()
paid_one_star_reviews

37

In [54]:
# paid 5-star reviews percentage
paid_one_star_percent = (paid_one_star_reviews / total_paid_reviews) * 100
paid_one_star_percent

5.7187017001545595

In [55]:
# unpaid 5-star reviews
unpaid_one_star_reviews = unpaid_df.filter(unpaid_df.star_rating == 1).count()
unpaid_one_star_reviews

14506

In [56]:
# unpaid 5-star reviews percentage
unpaid_one_star_percent = (unpaid_one_star_reviews / total_unpaid_reviews) * 100
unpaid_one_star_percent

19.572814485987614